In [15]:
import pandas as pd
tb1_final =pd.read_excel('提取结果.xlsx')

In [16]:
prompt = """
# 采购标的类型判断

## 任务目标
根据输入的采购标的名称，判断其所属标的类型，并按照规定的json格式输出。

## 标的类型
### 货物类
* 房屋和构筑物
* 设备
* 文物和陈列品
* 图书和档案
* 家具和用具
* 特种动植物
* 物资
* 无形资产

### 工程类
* 房屋施工
* 构筑物施工
* 施工工程准备
* 预制构件组装和装配
* 专业施工
* 安装工程
* 装修工程
* 修缮工程
* 工程设备租赁（带操作员）
* 其他建筑工程

### 服务类
* 科学研究和试验开发
* 教育服务
* 就业服务
* 医疗卫生服务
* 社会服务
* 文化、体育、娱乐服务
* 生态环境保护和治理服务
* 能源的生产和分配服务
* 农林牧渔服务
* 采矿业和制造业服务
* 工程管理服务
* 水利管理服务
* 公共设施管理服务
* 公园和游览景区服务
* 交通运输和仓储服务
* 信息技术服务
* 电信和其他信息传输服务
* 金融服务
* 专业技术服务
* 鉴证咨询服务
* 房地产服务
* 会议、展览、住宿和餐饮服务
* 商务服务
* 政府和社会资本合作服务
* 其他服务

## 判断规则
1. 仔细分析采购标的的名称和描述
2. 根据标的的性质确定其所属的大类（货物类/工程类/服务类）
3. 进一步确定具体的细分类别
4. 如果无法准确判断，选择最接近的类别

## 输出格式要求
输出标准JSON格式：
```json
{
    "类型": "具体的类型名称"
}
```

## 示例
输入：办公桌椅采购
输出：
```json
{
    "类型": "家具和用具"
}
```

现在请分析以下采购标的：{input_text}
"""

In [ ]:
from openai import OpenAI

# 填入你的实际 API Key
client = OpenAI(api_key="sk-", base_url="https://api.deepseek.com")
input_text = "全自动内镜清洗消毒机；干式荧光免疫分析仪；氩气高频电刀；食道阻抗-PH联合监测系统"
completion = client.chat.completions.create(
    model="deepseek-chat",
    messages=[
    {"role": "system","content": prompt},
    {
        "role": "user",
        "content": input_text
    }
    ],
    temperature=1,
    max_tokens=300,  # 修改为 max_tokens
    top_p=1,
    frequency_penalty=0,
    presence_penalty=0
)

# 打印响应内容
print(completion.choices[0].message.content)

{
    "类型": "设备"
}


In [9]:
import pandas as pd
import os
import json
import time
import logging
from datetime import datetime, timedelta
from concurrent.futures import ThreadPoolExecutor
from tqdm import tqdm
from openai import OpenAI

# -------------------------------
# JSON解析模块（独立模块）
# -------------------------------
def default_json_parser(content, idx=None):
    """
    默认的 JSON 解析器：
    清理输入内容后尝试解析 JSON，
    若成功则返回完整的字典，若失败返回空字典。
    """
    try:
        # 去除代码块标记，清理内容
        cleaned_content = content.replace('```json\n', '').replace('```', '').strip()
        parsed_result = json.loads(cleaned_content)
        return parsed_result
    except json.JSONDecodeError:
        if idx is not None:
            logging.warning(f"警告: 第 {idx} 行解析 JSON 失败")
        return {}
    except Exception as e:
        if idx is not None:
            logging.error(f"错误: 第 {idx} 行解析失败 - {str(e)}")
        return {}

# -------------------------------
# 限流处理器（控制请求频率）
# -------------------------------
class RateLimitedProcessor:
    def __init__(self):
        self.request_timestamps = []
        self.MAX_RPM = 500
        self.window_size = 60  # 60秒窗口

    def _clean_old_records(self, current_time):
        cutoff_time = current_time - timedelta(seconds=self.window_size)
        self.request_timestamps = [ts for ts in self.request_timestamps if ts > cutoff_time]

    def can_make_request(self):
        """检查是否可以发起新请求"""
        current_time = datetime.now()
        self._clean_old_records(current_time)
        if len(self.request_timestamps) >= self.MAX_RPM:
            return False
        self.request_timestamps.append(current_time)
        return True

# -------------------------------
# OpenAI文本处理器
# -------------------------------
class OpenAITextProcessor:
    def __init__(self, api_key=None, model=None, base_url=None, json_parser=None):
        self.client = OpenAI(api_key=api_key,base_url=base_url)
        self.model = model
        self.rate_limiter = RateLimitedProcessor()
        self.n_workers = 14  # 优化后的线程数
        # 如果未提供自定义解析器，则使用默认解析器
        self.json_parser = json_parser if json_parser is not None else default_json_parser

    def process_batch(self, df, text_column, prompt, batch_size=20, delay=1, json_parser=None):
        """
        批量处理文本，支持灵活的 JSON 解析。
        
        参数:
            df: 包含文本数据的 DataFrame
            text_column: 文本所在的列名
            prompt: 系统提示，用于 API 调用
            batch_size: 每个批次处理的文本条数
            delay: 每次请求后的延迟（秒）
            json_parser: 可选的自定义 JSON 解析器，若不传入则使用实例内的解析器
        
        返回:
            新的 DataFrame，包含原始数据及 API 返回结果（通过 JSON 解析获得的各字段）
        """
        parser = json_parser if json_parser is not None else self.json_parser
        results = []  # 保存每次请求解析后的结果（字典形式）

        def process_chunk(chunk_data):
            chunk_results = []
            for idx, text in chunk_data:
                # 限流检测：等待直到可以发送请求
                while not self.rate_limiter.can_make_request():
                    time.sleep(0.1)
                try:
                    response = self.client.chat.completions.create(
                        model=self.model,
                        messages=[
                            {"role": "system", "content": prompt},
                            {"role": "user", "content": text}
                        ],
                        temperature=0,
                        max_tokens=400
                    )
                    # 使用解析器处理响应内容，得到字典格式结果
                    parsed_result = parser(response.choices[0].message.content, idx)
                    chunk_results.append(parsed_result)
                    time.sleep(delay)
                except Exception as e:
                    logging.error(f"错误: 处理第 {idx} 行时发生异常: {str(e)}")
                    chunk_results.append({})
            return chunk_results

        # 将数据分成批次，保留行号信息
        chunks = [
            list(enumerate(df[text_column][i:i+batch_size]))
            for i in range(0, len(df), batch_size)
        ]

        with ThreadPoolExecutor(max_workers=self.n_workers) as executor:
            futures = list(tqdm(
                executor.map(process_chunk, chunks),
                total=len(chunks),
                desc="Processing batches"
            ))
            for chunk_results in futures:
                results.extend(chunk_results)

        # 将解析结果列表转为 DataFrame，并与原 DataFrame 合并
        df_result = df.copy().reset_index(drop=True)
        results_df = pd.json_normalize(results)
        df_result = pd.concat([df_result, results_df], axis=1)

        # 统计处理情况
        success_count = sum(1 for r in results if r)
        total_count = len(results)
        success_rate = (success_count / total_count) * 100 if total_count > 0 else 0
        logging.info(f"处理完成: 总数 {total_count}, 成功 {success_count}, 成功率 {success_rate:.2f}%")
        
        return df_result


In [ ]:
processor = OpenAITextProcessor(api_key="sk-", base_url="https://api.deepseek.com",model="deepseek-chat")
df_result = processor.process_batch(
    df=tb,
    text_column="主要标的名称",
    prompt=prompt,
    batch_size=5,
)

In [8]:
df_result=df_result[['主要标的名称','采购标的类型']]

In [9]:
df_result.to_excel("提取结果.xlsx", index=False)